In [1]:
import json
import numpy as np
import cv2
#import matplotlib.pyplot as plt
#import sys

In [2]:
dataset_name = "FIT3D"

data_root = "datasets"
subset = "train"
subj_name = "s03"
action_name = 'band_pull_apart'
camera_name = '50591643' # select from ['50591643', '58860488', '60457274', '65906101']
subject = 'w_markers' 
''' Fit3d has only the w_markers option'''

' Fit3d has only the w_markers option'

In [3]:
# returns a list of frames in RGB format
def read_video(vid_path):
    frames = []
    cap = cv2.VideoCapture(vid_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) )
    cap.release()
    frames = np.array(frames)
    return frames

# reads camera params
def read_cam_params(cam_path):
    with open(cam_path) as f:
        cam_params = json.load(f)
        for key1 in cam_params:
            for key2 in cam_params[key1]:
                cam_params[key1][key2] = np.array(cam_params[key1][key2]) 
    return cam_params

In [4]:
def read_data(data_root, dataset_name, subset, subj_name, action_name, camera_name, subject = "w_markers"):
    """ Builds the paths """
    vid_path = '%s/%s/%s/%s/videos/%s/%s.mp4' % (data_root, dataset_name, subset, subj_name, camera_name, action_name)
    cam_path = '%s/%s/%s/%s/camera_parameters/%s/%s.json' % (data_root, dataset_name, subset, subj_name, camera_name, action_name)
    j3d_path = '%s/%s/%s/%s/joints3d_25/%s.json' % (data_root, dataset_name, subset, subj_name, action_name)
    gpp_path = '%s/%s/%s/%s/gpp/%s.json' % (data_root, dataset_name, subset, subj_name, action_name)
    smplx_path = '%s/%s/%s/%s/smplx/%s.json' % (data_root, dataset_name, subset, subj_name, action_name)

    """ Read camera params """
    cam_params = read_cam_params(cam_path)

    with open(j3d_path) as f:
        j3ds = np.array(json.load(f)['joints3d_25'])
    seq_len = j3ds.shape[-3]
    with open(gpp_path) as f:
        gpps = json.load(f)
    with open(smplx_path) as f:
        smplx_params = json.load(f)
    frames = read_video(vid_path)[:seq_len]
    
    dataset_to_ann_type = {'fit3d': 'rep_ann'}
    
    ann_type = dataset_to_ann_type[dataset_name]
    annotations = None
    if ann_type:
        ann_path = '%s/%s/%s/%s/%s.json' % (data_root, dataset_name, subset, subj_name, ann_type)
        with open(ann_path) as f:
            annotations = json.load(f)
    
    return frames, j3ds, cam_params, gpps, smplx_params, annotations

In [ ]:
read_data(data_root,dataset_name,subset,subj_name,action_name,camera_name)